In [ ]:
from kite_trade import *
import threading
import time
import pandas as pd

In [ ]:
      
user_id = "QFW742"       # Login Id
password = "Dhinakar.s"      # Login password
twofa = "744557"         # Login Pin or TOTP

enctoken = get_enctoken(user_id, password, twofa)
kite = KiteApp(enctoken=enctoken)


In [33]:
# Circular buffer implementation
class CircularBuffer:
    def __init__(self, size):
        self.buffer = [None] * size
        self.size = size
        self.next_index = 0
    
    def add(self, data):
        self.buffer[self.next_index] = data
        self.next_index = (self.next_index + 1) % self.size
    
    def get_data(self):
        return self.buffer

# Initialize circular buffer
buffer_size = 1  # Adjust the size as needed
live_data_buffer = CircularBuffer(buffer_size)

# Function to continuously fetch live data and update circular buffer
def fetch_and_store_live_data(kite, buffer, duration, data_list):
    start_time = time.time()
    while True:
        elapsed_time = time.time() - start_time
        if elapsed_time >= duration:
            print("Duration reached. Stopping data fetching.")
            break
        
        live_data = kite.ltp(["NSE:NIFTY BANK", "NSE:NIFTY 50"])  # Fetch live data, adjust instruments as needed
        buffer.add(live_data)
        data_list.append(live_data)  # Append data to list
        time.sleep(1)  # Adjust the interval as needed

# Define duration for data fetching (in seconds)
duration = 10  # 10 seconds

data_list = []  # List to store live data

fetch_thread = threading.Thread(target=fetch_and_store_live_data, args=(kite, live_data_buffer, duration, data_list))
fetch_thread.start()

# Main loop to access live data
while fetch_thread.is_alive():
    # Access live data from the circular buffer whenever needed
    latest_data = live_data_buffer.get_data()
    print(latest_data)
    time.sleep(1)  # Adjust the interval as needed 

# Wait for the fetching thread to complete
fetch_thread.join()

# Convert data list to DataFrame
df = pd.DataFrame(data_list)

# Save DataFrame to Excel file
excel_file_path = r'C:\Users\Lenovo\kite\live_data.xlsx'
df.to_excel(excel_file_path, index=False)
print("Live data saved to:", excel_file_path)


[None]
[{'NSE:NIFTY 50': {'instrument_token': 256265, 'last_price': 22687.55}, 'NSE:NIFTY BANK': {'instrument_token': 260105, 'last_price': 49269.45}}]
[{'NSE:NIFTY 50': {'instrument_token': 256265, 'last_price': 22686.9}, 'NSE:NIFTY BANK': {'instrument_token': 260105, 'last_price': 49267.95}}]
[{'NSE:NIFTY 50': {'instrument_token': 256265, 'last_price': 22685.3}, 'NSE:NIFTY BANK': {'instrument_token': 260105, 'last_price': 49261.05}}]
[{'NSE:NIFTY 50': {'instrument_token': 256265, 'last_price': 22686.5}, 'NSE:NIFTY BANK': {'instrument_token': 260105, 'last_price': 49262.5}}]
[{'NSE:NIFTY 50': {'instrument_token': 256265, 'last_price': 22686.05}, 'NSE:NIFTY BANK': {'instrument_token': 260105, 'last_price': 49261.4}}]
[{'NSE:NIFTY 50': {'instrument_token': 256265, 'last_price': 22685.35}, 'NSE:NIFTY BANK': {'instrument_token': 260105, 'last_price': 49261.4}}]
[{'NSE:NIFTY 50': {'instrument_token': 256265, 'last_price': 22685.95}, 'NSE:NIFTY BANK': {'instrument_token': 260105, 'last_pric

In [ ]:
# Get instrument or exchange
#print(kite.instruments())
print(kite.instruments("NSE"))
print(kite.instruments("NFO"))

In [ ]:
# Basic calls
print(kite.margins())
print(kite.orders())
print(kite.positions())

In [ ]:
# Get Live Data
print(kite.ltp("NSE:RELIANCE"))
print(kite.ltp(["NSE:NIFTY 50", "NSE:NIFTY BANK"]))
print(kite.quote(["NSE:NIFTY BANK", "NSE:ACC", "NFO:NIFTY22SEPFUT"]))

In [ ]:
import time
while True:
    print(kite.quote(["NSE:NIFTY BANK"]))
    time.sleep(600)

In [ ]:
# Get Historical Data
import datetime
instrument_token = 424961
from_datetime = datetime.datetime.now() - datetime.timedelta(days=90)     # From last & days
to_datetime = datetime.datetime.now()
interval = "5minute"
print(kite.historical_data(instrument_token, from_datetime, to_datetime, interval, continuous=False, oi=False))

In [ ]:
instruments = kite.ltp('NSE:ITC') 
# Extract the instrument token
instrument_token = instruments['NSE:ITC']['instrument_token']
print(f"Instrument Token for ITC: {instrument_token}")

In [ ]:

import datetime

In [ ]:
instrument_token = 424961
from_datetime = datetime.datetime.now() - datetime.timedelta(days=7)
to_datetime = datetime.datetime.now()
interval = "5minute"

# Assuming kite.historical_data returns a list of dictionaries
historical_data = kite.historical_data(instrument_token, from_datetime, to_datetime, interval, continuous=False, oi=False)

# Convert the list of dictionaries to a Pandas DataFrame
df = pd.DataFrame(historical_data)

# Print or further manipulate the DataFrame as needed
print(df)


In [ ]:
#df.to_excel(r"C:\Users\Lenovo\kite\ITC_30D.csv")

In [ ]:
# Convert datetime columns to timezone-naive format
df['date'] = df['date'].dt.tz_localize(None)

# Export the DataFrame to an Excel file
excel_file_path = r'C:\Users\Lenovo\kite\ITC_30D.xlsx'
#df.to_excel(excel_file_path, index=False)

# Print or further manipulate the DataFrame as needed
print(df)
print(f'Data exported to {excel_file_path}')


In [ ]:
html_file_path=r'C:\Users\Lenovo\kite\ITC_30D.html'
df.to_html(html_file_path)

In [ ]:
#pip install TA_Lib‑0.4.0‑cp39‑cp39‑win_amd64.whl
#pip install TA-Lib-binary

In [ ]:
#conda install -c conda-forge ta-lib

In [ ]:
# Calculate Bollinger Bands
window = 20  # Adjust the window size as needed
df['MA'] = df['close'].rolling(window).mean()
df['std_dev'] = df['close'].rolling(window).std()
df['upper_band'] = df['MA'] + 2 * df['std_dev']
df['lower_band'] = df['MA'] - 2 * df['std_dev']

# Print or further manipulate the DataFrame as needed
print(df)

In [ ]:
# Calculate RSI
#rsi_period = 14  # Adjust the period as needed
#df['rsi'] = talib.RSI(df['close'].values, timeperiod=rsi_period)


# Calculate RSI
period = 14  # RSI period
delta = df['close'].diff()

gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()

rs = gain / loss
rsi = 100 - (100 / (1 + rs))

# Add RSI to DataFrame
df['RSI'] = rsi

# Print or further manipulate the DataFrame as needed
print(df)

In [ ]:
# Calculate Stochastic Oscillator
period = 14  # Adjust the period as needed
highest_high = df['high'].rolling(period).max()
lowest_low = df['low'].rolling(period).min()

df['%K'] = (df['close'] - lowest_low) / (highest_high - lowest_low) * 100
df['%D'] = df['%K'].rolling(3).mean()  # Smoothing with a 3-period moving average

# Print or further manipulate the DataFrame as needed
print(df)

In [ ]:
# Calculate MACD
short_period = 12  # Short-term moving average period
long_period = 26   # Long-term moving average period

# Calculate Short-term EMA (Exponential Moving Average)
ema_short = df['close'].ewm(span=short_period, min_periods=1).mean()

# Calculate Long-term EMA (Exponential Moving Average)
ema_long = df['close'].ewm(span=long_period, min_periods=1).mean()

# Calculate MACD line
macd_line = ema_short - ema_long

# Calculate Signal line (9-period EMA of the MACD line)
signal_line = macd_line.ewm(span=9, min_periods=1).mean()

# Calculate MACD histogram
macd_histogram = macd_line - signal_line

# Add MACD components to DataFrame
df['MACD Line'] = macd_line
df['Signal Line'] = signal_line
df['MACD Histogram'] = macd_histogram

# Print or further manipulate the DataFrame as needed
print(df)

In [ ]:
# Calculate Ichimoku Cloud components
conversion_line_period = 9
base_line_period = 26
leading_span_b_period = 52
displacement = 26  # The displacement for Chikou Span

# Calculate Conversion Line (Tenkan-sen)
conversion_line = (df['high'].rolling(window=conversion_line_period).max() + df['low'].rolling(window=conversion_line_period).min()) / 2

# Calculate Base Line (Kijun-sen)
base_line = (df['high'].rolling(window=base_line_period).max() + df['low'].rolling(window=base_line_period).min()) / 2

# Calculate Leading Span A
leading_span_a = ((conversion_line + base_line) / 2).shift(displacement)

# Calculate Leading Span B
leading_span_b = ((df['high'].rolling(window=leading_span_b_period).max() + df['low'].rolling(window=leading_span_b_period).min()) / 2).shift(displacement)

# Calculate Lagging Span (Chikou Span)
chikou_span = df['close'].shift(-displacement)

# Add Ichimoku Cloud components to DataFrame
df['Conversion Line'] = conversion_line
df['Base Line'] = base_line
df['Leading Span A'] = leading_span_a
df['Leading Span B'] = leading_span_b
df['Chikou Span'] = chikou_span

# Print or further manipulate the DataFrame as needed
print(df)

In [ ]:
html_file_path=r'C:\Users\Lenovo\kite\ITC_30D.html'
df.to_html(html_file_path)